In [1]:
import pandas as pd
import ta 
import optuna 
import time
from multiprocessing import Pool
from itertools import combinations, chain 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
class Position: 
    def __init__(self, timestamp, order_type, n_shares, stop_at, take_at, bought_at):
        self.timestamp = timestamp
        self.order_type = order_type
        self.n_shares = n_shares
        self.stop_at = stop_at
        self.take_at = take_at
        self.bought_at = bought_at

In [4]:
def powerset(s):
    return chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1))

In [5]:
def rsi_signals(data, rsi_window, rsi_upper, rsi_lower):
    indicator_rsi = ta.momentum.RSIIndicator(close=data["Close"], window=rsi_window)
    buy_signal = indicator_rsi.rsi() < rsi_lower
    sell_signal = indicator_rsi.rsi() > rsi_upper
    return buy_signal, sell_signal 

In [6]:
def roc_signals(data, roc_window, roc_upper, roc_lower):
    indicator_roc = ta.momentum.ROCIndicator(close=data["Close"], window=roc_window)
    buy_signal = indicator_roc.roc() > roc_lower
    sell_signal = indicator_roc.roc() < roc_upper
    return buy_signal, sell_signal

In [7]:
def tsi_signals(data, tsi_window_slow, tsi_window_fast, tsi_upper, tsi_lower):
    indicator_tsi = ta.momentum.TSIIndicator(close=data["Close"], 
                                             window_slow=tsi_window_slow, 
                                             window_fast=tsi_window_fast)
    buy_signal = indicator_tsi.tsi() > tsi_lower
    sell_signal = indicator_tsi.tsi() < tsi_upper
    return buy_signal, sell_signal

In [8]:
def stochastic_signals(data, stoch_window, stoch_smooth_window, stoch_upper, stoch_lower):
    indicator_stoch = ta.momentum.StochasticOscillator(close=data["Close"], high=data["High"], low=data["Low"], window=stoch_window, smooth_window=stoch_smooth_window)
    buy_signal = indicator_stoch.stoch() < stoch_lower
    sell_signal = indicator_stoch.stoch() > stoch_upper
    return buy_signal, sell_signal

In [9]:
def backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares):
    history = []
    active_operations = []
    cash = 1_000_000
    com = 1.25 / 100

    portfolio_value = []

    for i, row in data.iterrows():
        # close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp

        # check if we have enough cash
        if cash < (row.Close * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value.append(cash + asset_vals)
            continue

        # Apply buy signals
        if buy_signals.loc[i].any():
            active_operations.append({
                "timestamp": row.Timestamp,
                "bought": row.Close,
                "n_shares": n_shares,
                "stop_loss": row.Close * stop_loss,
                "take_profit": row.Close * take_profit
            })

            cash -= row.Close * (1 + com) * n_shares

        # Apply sell signals
        if sell_signals.loc[i].any():
            active_op_temp = []
            for operation in active_operations:
                if operation["take_profit"] < row.Close or operation["stop_loss"] > row.Close:
                    cash += (row.Close * operation["n_shares"]) * (1 - com)
                else:
                    active_op_temp.append(operation)
            active_operations = active_op_temp

        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value.append(cash + asset_vals)

    return portfolio_value


In [10]:
def optimize_file(file_path: str):
    data = pd.read_csv(file_path)
    strategies = list(powerset(["rsi", "roc", "tsi", "stoch"]))
    best_strat = None
    best_val = -1
    best_params = None
    
    for strat in strategies:
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda x: optimize(x, strat, data), n_trials=10)
        
        if study.best_value > best_val:
            best_val = study.best_value
            best_strat = strat
            best_params = study.best_params
            
    return {"file": file_path,
           "strat": best_strat, 
           "value": best_val,
           "params": best_params}

In [46]:
#if __name__ == "__main__":
#    with Pool(4) as p:
#        res = p.map(optimize_file, ["aapl_1m_train.csv",
#                                   "aapl_5m_train.csv",
#                                   "aapl_1h_train.csv",
#                                   "aapl_1d_train.csv"])
#        print(res)

In [11]:
def optimize(trial, strategy, data):
    stop_loss = trial.suggest_float("stop_loss", 0.00250, 0.05)
    take_profit = trial.suggest_float("take_profit", 0.00250, 0.05)
    n_shares = trial.suggest_int("n_shares", 5, 200)
    
    strat_params = {}
    
    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()
    
    if "rsi" in strategy:
        strat_params['rsi'] = {
            "rsi_window":trial.suggest_int("rsi_window", 5,100),
            "rsi_upper": trial.suggest_float("rsi_upper", 65, 95),
            "rsi_lower": trial.suggest_float("rsi_lower", 5, 35)
            
        }
        rsi_buy, rsi_sell = rsi_signals(data, **strat_params["rsi"])
        buy_signals["rsi"] = rsi_buy
        sell_signals["rsi"] = rsi_sell
    
    if "roc" in strategy:
        strat_params['roc'] = {
            "roc_window": trial.suggest_int("roc_window", 5,100),
            "roc_upper": trial.suggest_float("roc_upper", 0.8, 1.5),
            "roc_lower": trial.suggest_float("roc_lower", -2, -1)
            
        }
        roc_buy, roc_sell = roc_signals(data, **strat_params["roc"])
        buy_signals["roc"] = roc_buy
        sell_signals["roc"] = roc_sell
    
    if "tsi" in strategy:
        strat_params['tsi'] = {
            "tsi_window": trial.suggest_int("tsi_window", 5,100),
            "tsi_upper": trial.suggest_float("tsi_upper", 25, 45),
            "tsi_lower": trial.suggest_float("tsi_lower", -40, -20)
            
        }
        tsi_buy, tsi_sell = tsi_signals(data, **strat_params["tsi"])
        buy_signals["tsi"] = tsi_buy
        sell_signals["tsi"] = tsi_sell    
    
    
    if "stoch" in strategy:
        strat_params['stoch'] = {
            "stoch_window": trial.suggest_int("stoch_window", 5,100),
            "stoch_upper": trial.suggest_float("stoch_upper", 70, 90),
            "stoch_lower": trial.suggest_float("stoch_lower", 10, 30)
            
        }
        stoch_buy, stoch_sell = stoch_signals(data, **strat_params["stoch"])
        buy_signals["stoch"] = stoch_buy
        sell_signals["stoch"] = stoch_sell    
    
    return backtest(data, buy_signals, sell_signals, stop_loss, take_profit, n_shares)

In [13]:
one_minute = optimize_file("data/aapl_1m_train.csv")

[W 2024-03-05 17:55:44,058] Trial 0 failed with parameters: {'stop_loss': 0.0068554343784706945, 'take_profit': 0.04685011944859208, 'n_shares': 193, 'rsi_window': 40, 'rsi_upper': 66.99701402369293, 'rsi_lower': 10.404031534086803} because of the following error: The number of the values 395 did not match the number of the objectives 1.
[W 2024-03-05 17:55:44,058] Trial 0 failed with value [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 10

ValueError: No trials are completed yet.